In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
total = pd.read_csv('../input/rossmann-store-sales/train.csv')

In [ ]:
df = total.loc[total['Store']==2].copy()
df['date'] = pd.to_datetime(df['Date'])

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek
df['week'] = df['date'].dt.isocalendar().week.apply(lambda x: int(x))
df['StateHoliday'] = LabelEncoder().fit_transform(df['StateHoliday'].apply(lambda x: str(x)))
df['SchoolHoliday'] = LabelEncoder().fit_transform(df['SchoolHoliday'].apply(lambda x: str(x)))
df['Open'] = LabelEncoder().fit_transform(df['Open'].apply(lambda x: str(x)))
df['Promo'] = LabelEncoder().fit_transform(df['Promo'].apply(lambda x: str(x)))

In [ ]:
df

In [ ]:
train = df.loc[df['year']<2015].copy()
test = df.loc[df['year']==2015].copy()

In [ ]:
model = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.5,
    colsample_bytree=1,
    subsample=1,
    max_depth=5
)

In [ ]:
# model.fit(train[['Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek', 'week']], np.log1p(train['Sales']))
model.fit(train[['Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek', 'week']], train['Sales'])

In [ ]:
# test['pred'] = np.expm1(model.predict(test[['Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek', 'week']]))
test['pred'] = model.predict(test[['Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek', 'week']])
test.sort_values('date', inplace=True)

In [ ]:
test

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=test['date'], 
    y=test['Sales'],
    mode='lines',
    name='Sales',
))
fig.add_trace(go.Scatter(
    x=test['date'], 
    y=test['pred'],
    mode='lines',
    name='Prediction',
))
# fig.update_xaxes(type="log")
# fig.update_yaxes(type="log")
fig.show()

In [ ]:
test.loc[test['dayofweek']!=6, ['pred']].to_csv('pred.csv', index=False)

In [ ]:
test

In [ ]:
dump_list=model.get_booster().get_dump()

In [ ]:
print(dump_list[0])

In [ ]:
print(dump_list[1])

In [ ]:
print(dump_list[10])

In [ ]:
print(dump_list[100])

In [ ]:
print(dump_list[999])